In [56]:
import torch
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [57]:
data_path = r"C:\Users\VirtualUser\Desktop\Abraham_Internship\Abraham\Data"
data_path

'C:\\Users\\VirtualUser\\Desktop\\Abraham_Internship\\Abraham\\Data'

In [58]:
train_dir = data_path + "\\train"
val_dir = data_path + "\\validation"
train_dir

'C:\\Users\\VirtualUser\\Desktop\\Abraham_Internship\\Abraham\\Data\\train'

In [59]:
traincat = os.listdir(os.path.join(train_dir, "Cat"))
traindog = os.listdir(os.path.join(train_dir, "Dog"))
valcat = os.listdir(os.path.join(val_dir, "Cat"))
valdog = os.listdir(os.path.join(val_dir, "Dog"))
print("Train Cats:", len(traincat))
print("Train Dogs:", len(traindog))
print("Val Cats:", len(valcat))
print("Val Dogs:", len(valdog))

Train Cats: 9999
Train Dogs: 9999
Val Cats: 2500
Val Dogs: 2500


In [60]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

In [61]:
train_dataset = ImageFolder(train_dir, transform=train_transforms)
val_dataset = ImageFolder(val_dir, transform=val_transforms)

In [62]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
class_names = train_dataloader.dataset.classes
print(class_names)

['Cat', 'Dog']


In [63]:
print("No: of training images:", len(train_dataset))
print("No: of validation images:", len(val_dataset))
print("No: of training batches:", len(train_dataloader))
print("No: of validation batches:", len(val_dataloader))

No: of training images: 19998
No: of validation images: 5000
No: of training batches: 625
No: of validation batches: 157


In [64]:
i, j = next(iter(train_dataloader))
print(i.shape, j.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32])
